Find the optimal angle (toe-ing) to get the best in room response.
This is a failed experiment, please ignore.

In [ ]:
from math import log, sqrt
import numpy as np
import pandas as pd
import altair as alt

In [ ]:
import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.load import parse_all_speakers, parse_graphs_speaker, graph_melt
from spinorama.compute_normalize import normalize_mean, normalize_graph, normalize_cea2034
from spinorama.graph import graph_params_default, graph_freq, graph_spinorama
from spinorama.compute_cea2034 import compute_cea2034, estimated_inroom_HV, speaker_pref_rating

# df = parse_graphs_speaker('../datas/', 'Adam', 'Adam S2V', 'klippel')
# df = parse_graphs_speaker('../datas/', 'Genelec', 'Genelec 8341A', 'klippel')
# df = parse_graphs_speaker('../datas/', 'Polk', 'Polk T15', 'klippel')
df = parse_graphs_speaker('../datas/', 'KEF', 'KEF LS50', 'klippel')

In [ ]:
splH = df['SPL Horizontal_unmelted']
splV = df['SPL Vertical_unmelted']

In [ ]:
def turn(spl, angle):
    # return same spl DataFrame but turn by angle
    if spl is None:
        return None
    if angle == 0:
        return spl
    if angle%10 != 0:
        logging.error('angle must be a multiple of 10', angle)
        return None
    splT = pd.DataFrame({'Freq': spl.Freq})
    for col in spl.columns:
        if col is 'Freq':
            continue
        new_col = None
        if col == 'On Axis':
            new_col = '{0}°'.format(angle)
        else:
            new_angle = int(col[:-1])+angle
            if new_angle > 180:
                new_angle = -360+new_angle
            if new_angle < -180:
                new_angle = -new_angle+180
            if new_angle == 0:
                new_col = 'On Axis'
            else:
                new_col = '{0}°'.format(new_angle)
        # print('mapping {0} to {1} with angle {2}'.format(col, new_col, angle))
        splT.loc[:,new_col] = spl.loc[:,col]
        # sort columns in increasing angle order 
        def a2v(angle):
            if angle == 'Freq':
                return -1000
            elif angle == 'On Axis':
                return 0
            else:
                return int(angle[:-1])
    return splT.reindex(columns=sorted(splT.columns, key=lambda a: a2v(a)))
    

In [ ]:
splH50 = turn(splH, 50)
splV50 = turn(splV, 50)

In [ ]:
spin = graph_melt(compute_cea2034(splH, splV))
mean = normalize_mean(spin)
spin = normalize_cea2034(spin, mean)
eir  = graph_melt(estimated_inroom_HV(splH, splV))
eir  = normalize_graph(eir, mean)

spinH50V0 = graph_melt(compute_cea2034(splH50, splV))
spinH50V0 = normalize_cea2034(spinH50V0, mean)
eirH50V0  = graph_melt(estimated_inroom_HV(splH50, splV))
eirH50V0  = normalize_graph(eirH50V0, mean)

graph_spinorama(spin, graph_params_default) & graph_spinorama(spinH50V0, graph_params_default)
# graph_freq(eir, graph_params_default) & graph_freq(eirH50V0, graph_params_default)

In [ ]:
speaker_pref_rating(spin, eir)

In [ ]:
speaker_pref_rating(spinH50V0, eirH50V0)

In [ ]:
def find_best(splH, splV):
    spin = compute_cea2034(splH, splV)
    eir  = estimated_inroom_HV(splH, splV)
    ref = speaker_pref_rating(graph_melt(spin), graph_melt(eir))
    
    for angleH in (0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120):
        for angleV in (0, 10, 20):
            splHT = turn(splH, angleH)
            splVT = turn(splV, angleV)
            spinT = compute_cea2034(splHT, splVT)
            eirT  = estimated_inroom_HV(splHT, splVT)
            turned = speaker_pref_rating(graph_melt(spinT), graph_melt(eirT))
            print('H:{0:3d} V:{1:3d} nbd_on={2:.2f} sm_pir={3:.2f} pref_score={4:.2f}'.format(
                angleH, 
                angleV, 
                turned['nbd_on_axis'],
                turned['sm_pred_in_room'], 
                turned['pref_score']))

In [ ]:
find_best(splH, splV)

In [ ]:
dft = pd.DataFrame({'Freq': [0,1,2], 'On Axis': [0,0,0], '10°': [10, 10, 10], '20°': [20, 20, 20]})

In [ ]:
print(dft)
print(turn(dft, 10))